In [1]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import numpy as np
#import losses
import train_mnist_ot
from sklearn.datasets import load_svmlight_file
import pickle
tf.random.set_seed(666)
np.random.seed(666)
import ot
import dataloader

In [2]:
X_train, y_train, X_test, y_test = dataloader.load_data('mnist',False)

loading data


In [3]:
train_mnist_ot.train_baseline_ot_offline(X_train,y_train)

Epoch 1/5
1875/1875 [==============================] - 34s 7ms/step - loss: 0.1008 - sparse_categorical_accuracy: 0.9687
Epoch 2/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0361 - sparse_categorical_accuracy: 0.9885
Epoch 3/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0251 - sparse_categorical_accuracy: 0.9919
Epoch 4/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0163 - sparse_categorical_accuracy: 0.9947
Epoch 5/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0132 - sparse_categorical_accuracy: 0.9958


In [201]:
def FNN():
    inputs = Input((128,))
    FNN = tf.keras.models.Sequential([
        Dense(100, activation="relu"),
        Dense(80,activation="relu"),
        Dense(50,activation="relu"),
        Dense(6,activation="softmax")
    ])
    embeddings = FNN(inputs, training=True)
    FNN_network = Model(inputs, embeddings)
    #decay = tf.train
    FNN_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return FNN_network

In [300]:
model = FNN()
model.fit(X_train,y_train, batch_size=80,epochs=20)


Epoch 1/20
6/6 [==============================] - 0s 2ms/step - loss: 1.7082 - sparse_categorical_accuracy: 0.2360
Epoch 2/20
6/6 [==============================] - 0s 3ms/step - loss: 1.5755 - sparse_categorical_accuracy: 0.6584
Epoch 3/20
6/6 [==============================] - 0s 2ms/step - loss: 1.4455 - sparse_categorical_accuracy: 0.6067
Epoch 4/20
6/6 [==============================] - 0s 2ms/step - loss: 1.2803 - sparse_categorical_accuracy: 0.7213
Epoch 5/20
6/6 [==============================] - 0s 3ms/step - loss: 1.1155 - sparse_categorical_accuracy: 0.6944
Epoch 6/20
6/6 [==============================] - 0s 2ms/step - loss: 0.9612 - sparse_categorical_accuracy: 0.8315
Epoch 7/20
6/6 [==============================] - 0s 3ms/step - loss: 0.8268 - sparse_categorical_accuracy: 0.8382
Epoch 8/20
6/6 [==============================] - 0s 2ms/step - loss: 0.7118 - sparse_categorical_accuracy: 0.8584
Epoch 9/20
6/6 [==============================] - 0s 2ms/step - loss: 0.6154 - s

In [296]:
batch_length = [445, 1244, 1586,161,197,2300,3613,294,470,3600]
countnum = 0
counttotal = 0
count = 1
i = 0
while count < 10:
    model.evaluate(X_test[i:i+batch_length[count]], y_test[i:i+batch_length[count]])
    count+=1
    i+= batch_length[count]

39/39 [==============================] - 0s 1ms/step - loss: 3.5575 - sparse_categorical_accuracy: 0.3103


IndexError: list index out of range

In [277]:
ot_model=ot.da.EMDTransport()
results = np.array([])
target_size = 445
X_source = X_train
for i in range(0,len(X_test)):
    X_target = X_test[i*target_size : (i+1)*target_size]
    

    ot_model.fit(Xs=X_target, Xt=X_source)
    X_target_transform = ot_model.transform(Xs = X_target)
    labels =  np.argmax(model.predict(X_target),axis=1)

    #model.fit(X_target,labels, batch_size=80,epochs=3)
    print(labels)
    #X_source = X_target

    results = np.concatenate((results,labels))


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 5 5 5 4 4 4 4 1 1 1 2 2
 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 5 5 5 5 5 1 1 1 2
 1 2 1 2 1 1 1 1 1 1 

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 

[0 0 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1
 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 0 0 1
 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 1 0 0 1 1 0 0
 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0
 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1
 1 1 0 2 2 0 0 0 0 0 0 0 0 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2
 2 0 0 0 0 0 0 0 2 0 0 0 0 0 2 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 2 0 2 0
 2 0 2 0 2 0 2 0 2 0 0 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 0 2 0 2 0
 2 0 2 0 2 0 2 0 2 0 2 0 2 0 0 0 0 0 0 0 0 2 5 0 2 0 0 0 0 1 0 2 0 2 0 2 0
 2]
[0 2 0 2 0 2 0 2 0 2 0 2 0 1 0 2 0 0 0 2 0 2 0 2 0 1 0 2 0 2 0 1 0 2 0 2 0
 2 0 1 0 2 0 2 0 2 0 

[1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0
 0 0 0 1]


/usr/local/lib/python3.6/dist-packages/ot/lp/__init__.py:276: UserWarning: Problem infeasible. Check that a and b are in the simplex
  result_code_string = check_result(result_code)


ValueError: Expect x to be a non-empty array or dataset.

In [301]:
from collections import defaultdict
from sklearn.utils import shuffle
ot_model=ot.da.EMDTransport()
results = []
target_size = 445
X_source = X_train
sample_per_class = defaultdict(list)
accumulate_count = {1:0,2:0,3:0,4:0,5:0,0:0}
new_samples_dict = defaultdict(list)
BATCH_SIZE_ADAPT = 100


for i in range(len(y_train)):
    sample_per_class[y_train[i]].append(X_train[i])
count = 0
for i in range(0,len(X_test)):
    label =  np.argmax(model.predict(X_test[i].reshape(1,128)))
    if accumulate_count[label] >= len(sample_per_class[label]):
        accumulate_count[label] = 0
    sample_per_class[label][accumulate_count[label]] = X_test[i]
    accumulate_count[label] += 1
    count+=1
    if count > BATCH_SIZE_ADAPT:
        X_target = []
        y_target = []
        for key in sample_per_class:
            X_target = X_target + sample_per_class[key]
            y_target = np.concatenate((y_target,[key]*len(sample_per_class[key])))
        X_target = np.array(X_target)
        X_target,y_target = shuffle(X_target,y_target)
        ot_model.fit(Xs=X_target, Xt=X_source)
        X_target_transform = ot_model.transform(Xs = X_target)

        model.fit(X_target,y_target, batch_size=80,epochs=3)
        count = 0
        print(i)
    results.append(label)
    


Epoch 1/3
6/6 [==============================] - 0s 2ms/step - loss: 0.1415 - sparse_categorical_accuracy: 0.9663
Epoch 2/3
6/6 [==============================] - 0s 3ms/step - loss: 0.1255 - sparse_categorical_accuracy: 0.9685
Epoch 3/3
6/6 [==============================] - 0s 3ms/step - loss: 0.1173 - sparse_categorical_accuracy: 0.9708
100
Epoch 1/3
6/6 [==============================] - 0s 3ms/step - loss: 0.1071 - sparse_categorical_accuracy: 0.9708
Epoch 2/3
6/6 [==============================] - 0s 2ms/step - loss: 0.1009 - sparse_categorical_accuracy: 0.9708
Epoch 3/3
6/6 [==============================] - 0s 3ms/step - loss: 0.0958 - sparse_categorical_accuracy: 0.9730
201
Epoch 1/3
6/6 [==============================] - 0s 3ms/step - loss: 0.0923 - sparse_categorical_accuracy: 0.9865
Epoch 2/3
6/6 [==============================] - 0s 3ms/step - loss: 0.0832 - sparse_categorical_accuracy: 0.9910
Epoch 3/3
6/6 [==============================] - 0s 2ms/step - loss: 0.0766 - sp

6/6 [==============================] - 0s 3ms/step - loss: 0.0248 - sparse_categorical_accuracy: 0.9933
4746
Epoch 1/3
6/6 [==============================] - 0s 4ms/step - loss: 0.0316 - sparse_categorical_accuracy: 0.9888
Epoch 2/3
6/6 [==============================] - 0s 3ms/step - loss: 0.0257 - sparse_categorical_accuracy: 0.9910
Epoch 3/3
6/6 [==============================] - 0s 3ms/step - loss: 0.0252 - sparse_categorical_accuracy: 0.9865
4847
Epoch 1/3
6/6 [==============================] - 0s 3ms/step - loss: 0.0084 - sparse_categorical_accuracy: 0.9955
Epoch 2/3
6/6 [==============================] - 0s 2ms/step - loss: 0.0069 - sparse_categorical_accuracy: 1.0000
Epoch 3/3
6/6 [==============================] - 0s 3ms/step - loss: 0.0058 - sparse_categorical_accuracy: 1.0000
4948
Epoch 1/3
6/6 [==============================] - 0s 3ms/step - loss: 0.0104 - sparse_categorical_accuracy: 0.9955
Epoch 2/3
6/6 [==============================] - 0s 2ms/step - loss: 0.0093 - spars

6/6 [==============================] - 0s 2ms/step - loss: 0.0611 - sparse_categorical_accuracy: 0.9798
Epoch 3/3
6/6 [==============================] - 0s 2ms/step - loss: 0.0634 - sparse_categorical_accuracy: 0.9753
9493
Epoch 1/3
6/6 [==============================] - 0s 3ms/step - loss: 0.0349 - sparse_categorical_accuracy: 0.9978
Epoch 2/3
6/6 [==============================] - 0s 3ms/step - loss: 0.0319 - sparse_categorical_accuracy: 0.9910
Epoch 3/3
6/6 [==============================] - 0s 3ms/step - loss: 0.0329 - sparse_categorical_accuracy: 0.9910
9594
Epoch 1/3
6/6 [==============================] - 0s 3ms/step - loss: 0.0263 - sparse_categorical_accuracy: 0.9978
Epoch 2/3
6/6 [==============================] - 0s 2ms/step - loss: 0.0223 - sparse_categorical_accuracy: 0.9910
Epoch 3/3
6/6 [==============================] - 0s 2ms/step - loss: 0.0199 - sparse_categorical_accuracy: 0.9933
9695
Epoch 1/3
6/6 [==============================] - 0s 3ms/step - loss: 0.0339 - spars

In [302]:
batch_length = [445, 1244, 1586,161,197,2300,3613,294,470,3485]
countnum = 0
counttotal = 0
count = 1
i = 0
while count < 10:

    if results[i] == y_test[i]:
        countnum+=1
    counttotal += 1
    i+= 1
    if counttotal == batch_length[count]:
        print("Batch", count+1, " Accuracy is: ", countnum/counttotal)
        countnum = 0
        counttotal = 0
        count+= 1

Batch 2  Accuracy is:  0.572347266881029
Batch 3  Accuracy is:  0.4911727616645649
Batch 4  Accuracy is:  0.33540372670807456
Batch 5  Accuracy is:  0.2893401015228426
Batch 6  Accuracy is:  0.4291304347826087
Batch 7  Accuracy is:  0.27345696097425964
Batch 8  Accuracy is:  0.4965986394557823
Batch 9  Accuracy is:  0.39574468085106385
Batch 10  Accuracy is:  0.3764705882352941
